## _크롤링_

- resquests : 브라우저가 아닌 코드에서 서버로 리퀘스트를 보내는 것. 
    - 1.API 이용 : request를 던져서 JSON이라는 데이터 포맷(문자열)으로 데이터를 받는다. 문자열을 JSON object로 바꾸어주고 위해 파싱(parsing)을 한다. 그리고 JSON에서 필요한 데이터를 모으면 된다.
    - 2.HTML 코드 이용 : request를 던져서 HTML 코드를 문자열로 받는다. beautifulsoup4(bs4)를 이용해서 파싱(parsing)한다. 그러면 css-selector로 데이터를 모으면 된다. / 정적 웹페이지를 크롤링할 때 사용한다 / 
    - 3.Selenium 이용 : 프론트엔드 페이지 테스팅 도구. 직접 브라우저를 띄우고 css-selector를 이용해서 특정 데이터를 가져온다. / 동적 웹페이지를 크롤링할 때 사용한다
    - 4.scrapy 이용 : 웹 크롤링을 위해 만들어진 파이썬 패키지. request/response 방식. 크롤링 정책을 설정할 수 있다. 동적 페이지를 가져오기 위해서는 Selenium을 사용해야 한다. 



- 우선순위 : 1 > 2 > 3
    - JSON : 필요한 데이터만 집약되어 있다 
    - HTML : JSON에 비해 상대적으로 불필요한 데이터가 있다. request/response로 데이터만 가져온다.
    - Selenium : 브라우저를 띄우고 데이터를 가져오는데 자원이 소모된다.



- 기타 : 화면이 없는 서버에서 오류가 나지 않도록 하기 위해서
    - 크롬드라이버 헤드리스 : 화면을 띄우지 않은 상태에서 크롤링하는 것. Selenium이 이제 더는 phantom.JS를 지원하지 않음.
    - XVfb : 메모리 상 가상화면을 만들어서, 로컬의 화면을 그대로 가져와서 사용할 수 있다.

In [21]:
import requests
from bs4 import BeautifulSoup
import forecastio
import pandas as pd

### *실습*

- api json : 네이버 주식 데이터 크롤링
- api josn / forecastio : dark sky api로 날씨 데이터를 수집
- bs4 : 네이버 실시간 키워드, 다음 실시간 키워드 크롤링
- web file url로 파일을 다운로드 받는 방법

##### 네이버 주식데이터 가져오기

- 모바일 웹이 크롤링하기 더욱 용이하다

In [6]:
def make_url(pageSize=10, page=1):
    return "http://m.stock.naver.com/api/json/sise/siseListJson.nhn?\
    menu=market_sum&sosok=0&pageSize=" + str(pageSize) + "&page=" + str(page)
make_url(1458, 1)

'http://m.stock.naver.com/api/json/sise/siseListJson.nhn?    menu=market_sum&sosok=0&pageSize=1458&page=1'

In [25]:
# 데이터를 가져와서 판다스의 데이터프레임으로 만드는 함수
# 1.텍스트 그대로 가져오기
# 2.제이슨 오브젝트로 파싱해서 가져오기

def get_data(url):
    response = requests.get(url)
    json_info = response.json()
    companys = json_info["result"]["itemList"]
    df = pd.DataFrame(columns=["종목", "시세", "전일비", "등락율", "시가총액", "거래량"])
    
    for company in companys:
        df.loc[len(df)] = {
            "종목":company["nm"],
            "시세":company["nv"],
            "전일비":company["cv"],
            "등락율":company["cr"],
            "시가총액":company["mks"],
            "거래량":company["aq"],
        }  
    return df
#     print(response.text) # text 형태로 가져온다
#     print(json_info) # dict 형태로 가져온다
#     print(type(companys)) # List 형태로 가져온다

In [26]:
url = make_url(1500, 1)
df = get_data(url)
df.tail()

,종목,시세,전일비,등락율,시가총액,거래량
1453,KBSTAR 200에너지화학,9875,-10,-0.10,22,8033
1454,KBSTAR 200금융,9680,-175,-1.78,21,225
1455,ARIRANG 200동일가중,9940,-80,-0.80,15,50
1456,ARIRANG 200모멘텀,9770,-110,-1.11,15,28489
1457,ARIRANG 200로우볼,9665,-120,-1.23,14,28667


In [ ]:
# 뉴스 크롤링

In [28]:
def make_url(count=10, start=1):
    return "https://m.naver.com/api_airs_news?st=rec&display="\
+ str(count) + "&start=" + str(start)
make_url(100, 1)

'https://m.naver.com/api_airs_news?st=rec&display=100&start=1'

In [29]:
def get_data(url):
    response = requests.get(url)
    json_info = response.json()
    news = json_info["result"]["recommend"]
    df = pd.DataFrame(columns=["제목", "언론사", "시간"])
    
    for new in news:
        df.loc[len(df)] = {
            "제목":new["title"],
            "언론사":new["office_name"],
            "시간":new["dt_svc"],
        }  
    return df

In [30]:
url = make_url(100, 1)
df = get_data(url)
df.tail()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### _dark sky api_

- dark sky api : 위도와 경도를 입력하면 날씨 정보를 알려주는 api

In [31]:
FORECAST_TOKEN = "30dc208a4ebd307081cbb81003d5e455"

In [34]:
def forecast(lat, lng):
    url = "https://api.darksky.net/forecast/{}/{},{}"\
    .format(FORECAST_TOKEN, lat, lng)
    response = requests.get(url)
    json_info = response.json()
    return json_info["timezone"], json_info["hourly"]["summary"]

In [35]:
forecast(37.53, 127.04)

('Asia/Seoul', 'Mostly cloudy throughout the day.')

### _forecastio_

- 복잡한 api인 경우에는 패키지를 사용하는 것이 좋을 수 있다

In [36]:
def forecast2(lat, lng):
    forecast = forecastio.load_forecast(FORECAST_TOKEN, lat, lng)
    byHourly = forecast.hourly()
    return byHourly.summary
forecast2(37.53, 127.04)

'Mostly cloudy throughout the day.'

### _bs4_

- 네이버 키워드 랭킹 데이터 크롤링
- 다음 키워드 랭킹 데이터 크롤링